In [10]:
from IPython.display import display, HTML
import json
import os

import pandas as pd

In [1]:
METHODS = {
    "en": (
        "glossreader_v1",
        "mt0-definition-en-xl",
    ),
    "de": (
        "mt0-definition-en-xl",
        "glossreader_v1",
    ),
    "no1": (
        "mt0-definition-no-xl",
    ),
    "no2": (
        "mt0-definition-no-xl",
    ),
    "ru_ru": (
        "mt0-definition-ru-xl",
    ),
    "no1_en1": (
        "mt0-definition-en-xl",
    ),
    "no2_en2": (
        "mt0-definition-en-xl",
    ),
}

In [14]:
lang = "en"
annotations_path = os.path.expanduser("~/Downloads/english.json")
with open(annotations_path, "r", encoding="utf8") as f:
    annotations = json.load(f)
gloss_repo = os.path.expanduser("~/PycharmProjects/gloss-annotator")
predictions_folder = os.path.join(gloss_repo, "predictions")
annotations_dict = {}
for sample in annotations:
    # TODO don't uppercase my_text
    word_gloss = sample["data"]["my_text"]
    cluster_1 = sample["data"]["variants"][0]["value"]
    cluster_1_text = sample["data"]["variants"][0]["html"]
    cluster_2 = sample["data"]["variants"][1]["value"]
    cluster_2_text = sample["data"]["variants"][1]["html"]
    print(sample)
    sample_annotations = sample['annotations']
    for annotation in sample_annotations:
        cluster_true = annotation['result'][0]["value"]["choices"][0]
        annotations_dict[(word_gloss, cluster_1, cluster_2)] = (cluster_true, cluster_1_text, cluster_2_text)
        print(cluster_1)
        print(cluster_true)


{'id': 199, 'annotations': [{'id': 69, 'completed_by': 1, 'result': [{'value': {'choices': ['2']}, 'id': 'Ggaygh-xCO', 'from_name': 'selection', 'to_name': 'query', 'type': 'choices', 'origin': 'manual'}], 'was_cancelled': False, 'ground_truth': False, 'created_at': '2023-10-18T16:53:23.838044Z', 'updated_at': '2023-10-18T16:53:23.838077Z', 'draft_created_at': '2023-10-18T16:53:20.504729Z', 'lead_time': 47.802, 'prediction': {}, 'result_count': 0, 'unique_id': 'e22924d6-0aac-4182-b29b-f3083e05d68f', 'last_action': None, 'task': 199, 'project': 1, 'updated_by': 1, 'parent_prediction': None, 'parent_annotation': None, 'last_created_by': None}], 'file_upload': '5aba4498-label-studio-en.json', 'drafts': [], 'predictions': [], 'data': {'my_text': 'RECORD_NN: <b>A MEDIUM FOR RECORDING SOUND, ESPECIALLY A VINYL PHONOGRAPH AND GRAMOPHONE DISC.</b>', 'variants': [{'value': '2', 'html': '- I was making <b>records</b>, but they weren\'t getting played because there was no place to play them. "<br

In [16]:
annotations_dict

{('RECORD_NN: <b>A MEDIUM FOR RECORDING SOUND, ESPECIALLY A VINYL PHONOGRAPH AND GRAMOPHONE DISC.</b>',
  '2',
  '1'): ('2',
  '- I was making <b>records</b>, but they weren\'t getting played because there was no place to play them. "<br>- When the veteran rapper and actor LL Cool J\'s last Def Jam <b>record</b> tanked, he cut loose backstage at the MTV awards: " I think Jay-Z does a very good job, " he reportedly said. "<br>- What\'s more, sales of poetry on <b>records</b> are tuned to unprecedented volume.<br>- The <b>record</b> labels \' new services include MusicNet, backed by AOL Time Warner, BMG, and EMI; and PressPlay backed by Sony Music and Vivendi Universal.<br>- Constance starts to drink, stops, puts the needle on the <b>record</b> again, sits.<hr>',
  "- Columbus (Ohio) Quest Head coach: Brian Agler (31-9) <b>Record</b> last season: 31-9 Returning players: 6 Returning starters: 4 Probable starters F: Andrea Lloyd, 6-2 (Texas ' 87) F: Katie Smith, 5-11 (Ohio State ' 96) C: V

In [12]:
CLUSTER_NUMBER_COLUMN = 'cluster'

In [47]:
def print_wrongs(method):
    y_true, y_pred = [], []
    print(method)
    for word in os.listdir(os.path.join(predictions_folder,
                                        f"{method}/dwug_{lang}")):
        predictions_path = os.path.join(predictions_folder,
                                        f"{method}/dwug_{lang}/{word}/cluster_gloss.tsv")

        mapping_path = os.path.join(gloss_repo,
                                    f"wugs/label_studio_data/mappings/{lang}/{word}.tsv")
        try:
            mapping = pd.read_csv(mapping_path, sep="\t").astype(str)
        except FileNotFoundError:  # no annotation for this word e.g. if all its clusters are -1 etc.
            continue
        clusters_and_definitions = pd.read_csv(predictions_path, sep="\t")
        clusters_and_definitions[CLUSTER_NUMBER_COLUMN] = \
            clusters_and_definitions[CLUSTER_NUMBER_COLUMN].astype(str)
        clusters_and_definitions = clusters_and_definitions[
            clusters_and_definitions[CLUSTER_NUMBER_COLUMN] != "-1"]

        for row in clusters_and_definitions.iterrows():
            cluster_source, definition = row[1][CLUSTER_NUMBER_COLUMN], row[1][
                "gloss"]
            word_gloss = f"{word.upper()}: <b>{definition.upper()}</b>"
            cluster_filler = mapping[mapping[CLUSTER_NUMBER_COLUMN] == cluster_source][
                "wrong_cluster"]
            if cluster_filler.shape[0]:  # maybe cluster with < 3 usages
                cluster_filler = cluster_filler.iloc[0]
            else:
                continue
            
            annotation = annotations_dict.get((word_gloss, cluster_source, cluster_filler))
            ordering = 0
            if annotation is None:
                annotation = annotations_dict.get((word_gloss, cluster_filler, cluster_source))
                cluster_source_text = annotation[2]
                ordering = 1
            else:
                cluster_source_text = annotation[1]
                
            

            cluster_chosen = annotation[0] # number of cluster chosen by annotator
            
            
            
            
            if cluster_chosen != cluster_source:
                cluster_chosen_text = annotation[2]
                if ordering == 1:
                    cluster_chosen_text = annotation[1]
                display(HTML(word_gloss))
                print("Cluster chosen by annotator:")
                print(cluster_chosen) # -2 fits none, -3 fits both
                display(HTML(cluster_chosen_text))
                print("Cluster that generated the definition")
                print(cluster_source)
                display(HTML(cluster_source_text))
                print("-------------------------------------------------------------------")

In [48]:
print_wrongs("mt0-definition-en-xl")

mt0-definition-en-xl


Cluster chosen by annotator:
-2


Cluster that generated the definition
1


-------------------------------------------------------------------


Cluster chosen by annotator:
1


...

Cluster that generated the definition
2


-------------------------------------------------------------------


Cluster chosen by annotator:
2


Cluster that generated the definition
3


-------------------------------------------------------------------
